In [1]:
from bs4 import BeautifulSoup
import csv
import time
import requests
import pandas as pd
import openai

all_pmids = []
out = []

# 只显示第一页前50篇文章的信息
search_urls = ['https://pubmed.ncbi.nlm.nih.gov/?term=%28%22Information+processing+management%22%5BJournal%5D%29+AND+%28medical%5BTitle%2FAbstract%5D%29&sort=date&size=50']
for search_url in search_urls:
    
    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    pmids = soup.find_all('span', {'class' : 'docsum-pmid'})
    for p in pmids:
        p = p.get_text()
        all_pmids.append(p) if p not in all_pmids else print('project already in list, skipping')


for pmid in all_pmids:
    url = 'https://pubmed.ncbi.nlm.nih.gov/'+pmid
    response2 = requests.get(url)
    soup2 = BeautifulSoup(response2.content, 'html.parser')

    title = soup2.select('h1.heading-title')[0].text.strip()
    abstract = soup2.select('div.abstract-content.selected')[0].text.strip()
    keywords = soup2.select('div.abstract')[0].text.strip()
    keywords = keywords.rsplit('\n',1)[1]
    keywords = 'keywords:' + keywords
        
    data = {'title': title, 'pmid': pmid, 'url':url, 'abstract':abstract, 'keywords':keywords}
    time.sleep(3)
    out.append(data)

df = pd.DataFrame(out)
df['t_chinese'] = '0'
df['abs_chinese'] = '0'
df['key_chinese'] = '0'


In [2]:
df

,title,pmid,url,abstract,keywords,t_chinese,abs_chinese,key_chinese
0,Developing a socio-computational approach to e...,36567973,https://pubmed.ncbi.nlm.nih.gov/36567973,As the novel coronavirus (COVID-19) continues ...,keywords: COVID-19; Pandemic; Social medi...,0,0,0
1,Characterizing the patterns of China's policie...,33678941,https://pubmed.ncbi.nlm.nih.gov/33678941,"Since the beginning of 2020, the Chinese gover...",keywords: Bibliometrics; COVID-19; China;...,0,0,0
2,Turn to the Internet First? Using Online Medic...,33519039,https://pubmed.ncbi.nlm.nih.gov/33519039,The surveillance and forecast of newly confirm...,keywords: COVID-19; eHealth; online medic...,0,0,0
3,Promoting users' intention to share online hea...,32834400,https://pubmed.ncbi.nlm.nih.gov/32834400,"Nowadays, it is a common practice for healthca...",keywords: Confirmation bias; Content vale...,0,0,0


In [3]:


# 用openai的api对df['abstract']生成摘要翻译并存入df['abs_chinese']
import openai
openai.api_key = "xxxxxxxxxx"  # 填入自己的api key
model_engine = 'text-davinci-003'

for i in range(len(df)):
    prompt = "please translate the following to simplified chinese:"
    prompt += df['abstract'][i]
    completion = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    df['abs_chinese'][i] = completion.choices[0].text




In [4]:
for i in range(len(df)):
    prompt = "please translate the following to simplified chinese:"
    prompt += df['title'][i]
    completion = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    df['t_chinese'][i] = completion.choices[0].text

In [5]:
for i in range(len(df)):
    prompt = "please translate the following to simplified chinese:"
    prompt += df['keywords'][i]
    completion = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    df['key_chinese'][i] = completion.choices[0].text

In [6]:
df

,title,pmid,url,abstract,keywords,t_chinese,abs_chinese,key_chinese
0,Developing a socio-computational approach to e...,36567973,https://pubmed.ncbi.nlm.nih.gov/36567973,As the novel coronavirus (COVID-19) continues ...,keywords: COVID-19; Pandemic; Social medi...,\n\n开发一种社会计算方法来研究YouTube上关于COVID-19讨论中毒性传播和调节的情况。,\n\n随着新冠肺炎(COVID-19)以前所未有的速度肆虐全球，医学专家的正式建议正被社交...,\n\nCOVID-19;大流行;社交媒体;社交网络分析;主题建模;毒性分析;YouTube。
1,Characterizing the patterns of China's policie...,33678941,https://pubmed.ncbi.nlm.nih.gov/33678941,"Since the beginning of 2020, the Chinese gover...",keywords: Bibliometrics; COVID-19; China;...,\n\n对中国政策应对新冠肺炎疫情的模式进行描述：一项文献计量研究,\n\n2020年初以来，中国政府实施了大量政策来防控新冠肺炎疫情。本研究试图揭示和描述中国...,\n\n统计学图书馆学；COVID-19；中国；疫情防控；政策分析。
2,Turn to the Internet First? Using Online Medic...,33519039,https://pubmed.ncbi.nlm.nih.gov/33519039,The surveillance and forecast of newly confirm...,keywords: COVID-19; eHealth; online medic...,\n\n首先使用网络? 利用在线医疗行为数据预测新冠疫情趋势,\n\n在公共卫生紧急情况下，新增确诊病例的监测和预测对动员医疗资源和促进政策制定至关重要。...,\n\nCOVID-19; 电子健康; 在线医疗咨询; 大流行; 远程医疗。
3,Promoting users' intention to share online hea...,32834400,https://pubmed.ncbi.nlm.nih.gov/32834400,"Nowadays, it is a common practice for healthca...",keywords: Confirmation bias; Content vale...,\n\n提升用户分享网络健康文章在社交媒体上的意向：确认偏见的作用,\n\n现今，医疗保健专业通过在社交媒体上发布健康文章来传播医学知识是一种常见做法。然而，推...,\n\n确认偏见；内容情感；健康信息行为；社交媒体；电子健康素养。


In [7]:
df[['t_chinese','abs_chinese','key_chinese']] = df[['t_chinese','abs_chinese','key_chinese']].applymap(lambda x: x.rsplit('\n',1)[1])

In [8]:
df

,title,pmid,url,abstract,keywords,t_chinese,abs_chinese,key_chinese
0,Developing a socio-computational approach to e...,36567973,https://pubmed.ncbi.nlm.nih.gov/36567973,As the novel coronavirus (COVID-19) continues ...,keywords: COVID-19; Pandemic; Social medi...,开发一种社会计算方法来研究YouTube上关于COVID-19讨论中毒性传播和调节的情况。,随着新冠肺炎(COVID-19)以前所未有的速度肆虐全球，医学专家的正式建议正被社交媒体平台...,COVID-19;大流行;社交媒体;社交网络分析;主题建模;毒性分析;YouTube。
1,Characterizing the patterns of China's policie...,33678941,https://pubmed.ncbi.nlm.nih.gov/33678941,"Since the beginning of 2020, the Chinese gover...",keywords: Bibliometrics; COVID-19; China;...,对中国政策应对新冠肺炎疫情的模式进行描述：一项文献计量研究,2020年初以来，中国政府实施了大量政策来防控新冠肺炎疫情。本研究试图揭示和描述中国应对新冠...,统计学图书馆学；COVID-19；中国；疫情防控；政策分析。
2,Turn to the Internet First? Using Online Medic...,33519039,https://pubmed.ncbi.nlm.nih.gov/33519039,The surveillance and forecast of newly confirm...,keywords: COVID-19; eHealth; online medic...,首先使用网络? 利用在线医疗行为数据预测新冠疫情趋势,在公共卫生紧急情况下，新增确诊病例的监测和预测对动员医疗资源和促进政策制定至关重要。随着互联...,COVID-19; 电子健康; 在线医疗咨询; 大流行; 远程医疗。
3,Promoting users' intention to share online hea...,32834400,https://pubmed.ncbi.nlm.nih.gov/32834400,"Nowadays, it is a common practice for healthca...",keywords: Confirmation bias; Content vale...,提升用户分享网络健康文章在社交媒体上的意向：确认偏见的作用,现今，医疗保健专业通过在社交媒体上发布健康文章来传播医学知识是一种常见做法。然而，推广用户分...,确认偏见；内容情感；健康信息行为；社交媒体；电子健康素养。


In [9]:
df['key_chinese'] = df['key_chinese'].apply(lambda x: '关键词:'+str(x))

In [10]:
df

,title,pmid,url,abstract,keywords,t_chinese,abs_chinese,key_chinese
0,Developing a socio-computational approach to e...,36567973,https://pubmed.ncbi.nlm.nih.gov/36567973,As the novel coronavirus (COVID-19) continues ...,keywords: COVID-19; Pandemic; Social medi...,开发一种社会计算方法来研究YouTube上关于COVID-19讨论中毒性传播和调节的情况。,随着新冠肺炎(COVID-19)以前所未有的速度肆虐全球，医学专家的正式建议正被社交媒体平台...,关键词:COVID-19;大流行;社交媒体;社交网络分析;主题建模;毒性分析;YouTube。
1,Characterizing the patterns of China's policie...,33678941,https://pubmed.ncbi.nlm.nih.gov/33678941,"Since the beginning of 2020, the Chinese gover...",keywords: Bibliometrics; COVID-19; China;...,对中国政策应对新冠肺炎疫情的模式进行描述：一项文献计量研究,2020年初以来，中国政府实施了大量政策来防控新冠肺炎疫情。本研究试图揭示和描述中国应对新冠...,关键词:统计学图书馆学；COVID-19；中国；疫情防控；政策分析。
2,Turn to the Internet First? Using Online Medic...,33519039,https://pubmed.ncbi.nlm.nih.gov/33519039,The surveillance and forecast of newly confirm...,keywords: COVID-19; eHealth; online medic...,首先使用网络? 利用在线医疗行为数据预测新冠疫情趋势,在公共卫生紧急情况下，新增确诊病例的监测和预测对动员医疗资源和促进政策制定至关重要。随着互联...,关键词:COVID-19; 电子健康; 在线医疗咨询; 大流行; 远程医疗。
3,Promoting users' intention to share online hea...,32834400,https://pubmed.ncbi.nlm.nih.gov/32834400,"Nowadays, it is a common practice for healthca...",keywords: Confirmation bias; Content vale...,提升用户分享网络健康文章在社交媒体上的意向：确认偏见的作用,现今，医疗保健专业通过在社交媒体上发布健康文章来传播医学知识是一种常见做法。然而，推广用户分...,关键词:确认偏见；内容情感；健康信息行为；社交媒体；电子健康素养。


In [11]:

df.to_csv(r"C:/Users/XC/Desktop/pubmed_articles.csv")


In [12]:
import csv
import pandas as pd
df = pd.read_csv(r"C:/Users/XC/Desktop/pubmed_articles.csv",index_col='Unnamed: 0')

In [13]:
df

,title,pmid,url,abstract,keywords,t_chinese,abs_chinese,key_chinese
0,Developing a socio-computational approach to e...,36567973,https://pubmed.ncbi.nlm.nih.gov/36567973,As the novel coronavirus (COVID-19) continues ...,keywords: COVID-19; Pandemic; Social medi...,开发一种社会计算方法来研究YouTube上关于COVID-19讨论中毒性传播和调节的情况。,随着新冠肺炎(COVID-19)以前所未有的速度肆虐全球，医学专家的正式建议正被社交媒体平台...,关键词:COVID-19;大流行;社交媒体;社交网络分析;主题建模;毒性分析;YouTube。
1,Characterizing the patterns of China's policie...,33678941,https://pubmed.ncbi.nlm.nih.gov/33678941,"Since the beginning of 2020, the Chinese gover...",keywords: Bibliometrics; COVID-19; China;...,对中国政策应对新冠肺炎疫情的模式进行描述：一项文献计量研究,2020年初以来，中国政府实施了大量政策来防控新冠肺炎疫情。本研究试图揭示和描述中国应对新冠...,关键词:统计学图书馆学；COVID-19；中国；疫情防控；政策分析。
2,Turn to the Internet First? Using Online Medic...,33519039,https://pubmed.ncbi.nlm.nih.gov/33519039,The surveillance and forecast of newly confirm...,keywords: COVID-19; eHealth; online medic...,首先使用网络? 利用在线医疗行为数据预测新冠疫情趋势,在公共卫生紧急情况下，新增确诊病例的监测和预测对动员医疗资源和促进政策制定至关重要。随着互联...,关键词:COVID-19; 电子健康; 在线医疗咨询; 大流行; 远程医疗。
3,Promoting users' intention to share online hea...,32834400,https://pubmed.ncbi.nlm.nih.gov/32834400,"Nowadays, it is a common practice for healthca...",keywords: Confirmation bias; Content vale...,提升用户分享网络健康文章在社交媒体上的意向：确认偏见的作用,现今，医疗保健专业通过在社交媒体上发布健康文章来传播医学知识是一种常见做法。然而，推广用户分...,关键词:确认偏见；内容情感；健康信息行为；社交媒体；电子健康素养。


In [39]:
from docx import Document
from docx.shared import Pt # 设置字号
from docx.shared import RGBColor # 设置字体颜色

document = Document()

for i in range(len(df)):
    title = document.add_heading(str(i+1),level=1)
    run1 = title.add_run(df['title'][i])  
    run.style.font.name = 'Times New Roman' # 设置中文字体前面要有u
    run.font.bold = True # 设置粗体
    run2 = title.add_run(df['t_chinese'][i])
    run2.style.font.name = u'宋体' # 设置中文字体前面要有u
    run2.font.italic = True # 设置斜体
    run2.font.color.rgb = RGBColor(250,0,0) # 设置字体颜色

    body = document.add_paragraph(df['abs_chinese'][i])
    body = body.add_run('')
    body.font.name = u'宋体' #设置字体
    body.font.size = Pt(10.5) # 设置字号

    tail = document.add_paragraph(df['key_chinese'][i])
    tail = tail.add_run('')
    tail.font.name = u'宋体' #设置字体
    tail.font.size = Pt(10.5) # 设置字号


# 保存
document.save(r"C:/Users/XC/Desktop/pubmed_articles.docx")